In [2]:
import tensorflow_datasets as tfds

In [5]:
import tensorflow as tf

In [6]:
(train_data, test_data), ds_info = tfds.load('food101', split=['train', 'validation'], shuffle_files=True, as_supervised=True, with_info=True)

In [ ]:
ds_info.features

In [ ]:
ds_info.features['label'].names[:10]

In [ ]:
train_data.take(1)

In [ ]:
for image, label in train_data.take(1):
    print(image.shape)
    print(image.dtype)

In [7]:
def preprocess(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32)
    return image, label

In [8]:
train_data = train_data.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(32).prefetch(tf.data.AUTOTUNE)

test_data = test_data.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

In [ ]:
len(train_data), len(test_data)

In [ ]:
train_data.take(1).get_single_element()[1]

In [9]:
from keras import mixed_precision

In [10]:
mixed_precision.set_global_policy('mixed_float16')

In [11]:
mixed_precision.global_policy()

<DTypePolicy "mixed_float16">

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(101)(x)
outputs = tf.keras.layers.Activation("softmax", dtype=tf.float32)(x)

model1 = tf.keras.Model(inputs, outputs)

model1.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_top = model1.fit(train_data, epochs=50, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-30:]:
    layer.trainable = True

model1.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0005), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft1 = model1.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-60:]:
    layer.trainable = True

model1.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0001), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft2 = model1.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

In [ ]:
model1.evaluate(test_data, steps=len(test_data))

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(101)(x)
outputs = tf.keras.layers.Activation("softmax", dtype=tf.float32)(x)

model2 = tf.keras.Model(inputs, outputs)

model2.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_top = model2.fit(train_data, epochs=50, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-30:]:
    layer.trainable = True

model2.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.001), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft1 = model2.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-60:]:
    layer.trainable = True

model2.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0005), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft2 = model2.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

In [ ]:
model2.evaluate(test_data, steps=len(test_data))

Не перевищувати розмір batch_size = 32. Не вистачає пам'яті

In [ ]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(101, kernel_regularizer=tf.keras.regularizers.l2(0.01), bias_regularizer=tf.keras.regularizers.l2(0.01))(x)
outputs = tf.keras.layers.Activation("softmax", dtype=tf.float32)(x)

model3 = tf.keras.Model(inputs, outputs)

model3.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_top = model3.fit(train_data, epochs=50, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-30:]:
    layer.trainable = True

model3.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0005), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft1 = model3.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

for layer in base_model.layers[-60:]:
    layer.trainable = True

model3.compile(loss="sparse_categorical_crossentropy", optimizer=tf.keras.optimizers.Adam(0.0001), metrics=["accuracy"])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", mode="max", patience=3, verbose=1, restore_best_weights=True)

history_ft2 = model3.fit(train_data, epochs=100, validation_data=test_data, steps_per_epoch=len(train_data), validation_steps=len(test_data), callbacks=[early_stopping])

Epoch 1/50


I0000 00:00:1739013631.344089   29197 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1739013642.350684   29197 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2368/2368 ━━━━━━━━━━━━━━━━━━━━ 98s 33ms/step - accuracy: 0.3831 - loss: 3.8301 - val_accuracy: 0.5850 - val_loss: 2.9062
Epoch 2/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 57s 24ms/step - accuracy: 0.4874 - loss: 3.1851 - val_accuracy: 0.5829 - val_loss: 2.8994
Epoch 3/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 57s 24ms/step - accuracy: 0.4909 - loss: 3.1730 - val_accuracy: 0.5861 - val_loss: 2.9030
Epoch 4/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 57s 24ms/step - accuracy: 0.4896 - loss: 3.1738 - val_accuracy: 0.5857 - val_loss: 2.9021
Epoch 5/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 56s 23ms/step - accuracy: 0.4866 - loss: 3.1850 - val_accuracy: 0.5823 - val_loss: 2.9054
Epoch 6/50
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 56s 23ms/step - accuracy: 0.4908 - loss: 3.1779 - val_accuracy: 0.5855 - val_loss: 2.9006
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 3.
Epoch 1/100
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 121s 40ms/step - accuracy: 0.5359 - loss: 2.4811 - val_accuracy: 0.7296 - val_loss: 1.3695
Ep